In [1]:
%load_ext autoreload
%autoreload 2

import os
import pathlib
cwd = pathlib.Path.cwd()
if cwd.name != 'ryn':
    print('changing directory')
    os.chdir(cwd.parent)

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

changing directory


In [3]:
from ryn.graphs import split
from ryn.graphs import loader

g = loader.load_graphs_from_uri('oke.fb15k237-trainvalidtest')[0]
print(f'loaded {g.str_stats}')

rels = split.Relation.from_graph(g)
rels.sort(key=lambda rel: rel.ratio)
print(f'retrieved {len(rels)} relations')

loaded ryn graph: oke.fb15k237-trainvalidtest
  nodes: 14541
  edges: 310116 (237 types)
  degree:
    mean 42.65
    median 26

retrieved 237 relations


In [ ]:
from tabulate import tabulate

rows = [(r.r, r.ratio, len(r.hs), len(r.ts), r.name) for r in rels]

N = 5

print(f'first {N}')
print(tabulate(rows[:N]))

print(f'mid {N}')
m = len(rows) / 2
print(tabulate(rows[int(m-N/2):int(m+N/2)]))

print(f'last {N}')
print(tabulate(rows[-N:]))

In [ ]:
import ryn
from ryn.common import plotter

from functools import partial

path = ryn.ENV.CACHE_DIR / 'notes.graph.split' / f'{g.name}.ratio'
prob = partial(split._prob, a=cfg.prob_a, o=cfg.prob_o, s=cfg.prob_s)

plt = plotter.Plotter(title=f'Ratio Distribution {g.name}', xlabel='Relation', ylabel='Ratio', fname=str(path))
plt.ax.scatter(range(len(rels)), [r.ratio for r in rels], color=plotter.CLR[0], s=1)
plt.ax.scatter(range(len(rels)), [prob(x) for x in range(len(rels))], color=plotter.CLR[1], s=1)

print()
plt.plot()
print()

print(tabulate(rows[100:105]))
print([f'{prob(x):.2f}' for x in range(80,90)])

In [ ]:
greedy_cfg = split.GreedyConfig(
    owcw_split=.7, trainvalid_split=.7, pathname='greedy',
    prob_a=3, prob_o=len(rels) / 2, prob_s=1/20)

_p = int(greedy_cfg.owcw_split * 100)
print(f'creating a greedy split of {_p}% ow and {100 - _p}% cw data')

seeds = [30061990, 8051991, 25031990, 2041992]
seeds = [30061990, ]
split.create_greedy(g, greedy_cfg, rels, seeds)
print('done')

creating a greedy split of 70% ow and 30% cw data
0 25 3055
1 43 3177
2 120 677


3 85 2512
4 118 344
5 33 320
6 11 1225
7 110 180
8 34 1022
9 60 395
10 195 131
11 167 109
12 206 106
13 234 100
14 91 106
15 161 381


16 5 1596
17 154 973
18 4 1607
19 213 138
20 208 131
21 115 298
22 219 163
23 108 299
24 200 111
25 166 317
26 63 591
27 128 246
28 80 367
29 129 152
30 16 1
31 87 70
32 172 294
33 232 93


34 73 207
35 209 32
36 217 100
37 48 869
38 113 125
39 212 148
40 139 129
41 90 250
42 168 125
43 78 40


44 26 4484
45 0 253
46 14 2154
47 17 9370


...

200 35 654
201 54 80
202 226 86
203 194 153
204 229 154
205 196 90
206 152 391
207 144 75
208 57 496
209 178 125


210 192 182
211 188 174
212 228 78
213 53 58
214 123 215
215 58 1128
216 119 142


217 45 748
218 175 60
219 9 2711
220 75 3887
221 141 264
222 164 107
223 190 91
224

 97 309
225 3 4326


226 8 8461
227 77 692
228 56 403
229 64 403
230 81 378
231 114 28
232 146 388
233 180 102
